In [1]:
!python3 --version

Python 3.7.11


In [2]:
!pip install -U pytube
!pip install -U get-video-properties

     |████████████████████████████████| 56 kB 3.5 MB/s 
     |████████████████████████████████| 45.1 MB 26 kB/s 


In [3]:
import os
import pytube
import sys
import pytube
import pandas as pd
import urllib.request
from videoprops import get_video_properties

In [4]:
PROJECT_DIR = r"/content/drive/MyDrive/Mini Project II/Dataset/"
TRAINING_DIR = os.path.join(PROJECT_DIR,'Training')
TEST_DIR = os.path.join(PROJECT_DIR,"Test")
DATASET_DIR = os.path.join(TEST_DIR,'videos')

In [5]:
print(DATASET_DIR)

/content/drive/MyDrive/Mini Project II/Dataset/Test/videos


### Download Test Dataset Videos

In [ ]:
def downloadYoutube(filename,url):
    try:
        yt = pytube.YouTube(url)
        video = yt.streams.get_highest_resolution()
        out_file = video.download(f'{DATASET_DIR}/')
        ext = out_file.split('.')[-1]
        yt_id_filename = '.'.join([filename,ext])
        os.rename(out_file,f'{DATASET_DIR}/'+yt_id_filename)
        print(yt_id_filename + " download successful")
    except Exception as e:
        print('Error downloading {0} : {1}'.format(filename,e))

In [ ]:
def downloadIMDB(filename,url):
    videoplayerlink=url
    html=""
    with urllib.request.urlopen(videoplayerlink) as response:
        html = response.read()
    html=str(html)
    script = html.split('"video/mp4')
    videoLink=script[1].split('},{')[0].split('"')[4][0:-2]

    urllib.request.urlretrieve(videoLink,os.path.join(DATASET_DIR,f'{filename}.mp4'))

In [8]:
if not os.path.exists(DATASET_DIR):
    os.mkdir(DATASET_DIR)
final_videos=[]

csvFile=pd.read_csv('/content/drive/MyDrive/Mini Project II/Dataset/Test/Dataset_ComicMischief_Test_Scenes.csv')  

def compute_frame_rate_from_string_fraction(fraction):
    num,den = fraction.split( '/' )
    result= float(num)/float(den)
    return result

In [ ]:
len(csvFile['Video ID'])

237

In [ ]:
len(set(csvFile['Video ID']))

48

In [ ]:
import time

#### Main Script - Downloading Videos from both Youtube and IMDB
totalSuccessfullAttempts=0
missingVideos=[]
for i,URL in enumerate(csvFile["Video URL"]):
    time.sleep(0.5)
    if(i==0 or (csvFile["Video URL"][i-1] not in URL)):
        try:
            if not os.path.exists(DATASET_DIR+f'{csvFile["Video ID"][i]}.mp4'):
                if "youtube" in URL:
                    downloadYoutube(csvFile["Video ID"][i],URL)
            
                if "imdb" in URL:
                    # pass
                    downloadIMDB(csvFile["Video ID"][i],URL)    
            props = get_video_properties(os.path.join(DATASET_DIR,f"{csvFile['Video ID'][i]}.mp4"))
            print(f"{i}. Video ID:" +"\t"+csvFile["Video ID"][i] +"\t URL: \t"+ URL+"\t"+"Codec:"+"\t"+  props['codec_name']+"\t"+ "Resolution:"+"\t"+ str(props['width']),"x",str(props['height']) +"\t"+ "Avg Frame rate:" +"\t"+ str(compute_frame_rate_from_string_fraction(props['avg_frame_rate'])))
            totalSuccessfullAttempts+=1
        except Exception as e:
            print(f"{i}. Video ID: " +"\t"+csvFile["Video ID"][i] +"\t URL: \t"+ URL+"\t"+ "Unable to download video")
            print('Exception : ',e)
            missingVideos.append((csvFile['Video ID'][i],URL))
print("Total Successfull Attempts = ", totalSuccessfullAttempts)
print("Missing Videos = ", missingVideos)

0. Video ID:	tt2872718	 URL: 	 https://www.imdb.com/videoplayer/vi4179799833	Codec:	h264	Resolution:	854 x 480	Avg Frame rate:	23.976023976023978
2. Video ID:	tt2788710	 URL: 	 https://www.imdb.com/videoplayer/vi1114222361	Codec:	h264	Resolution:	854 x 480	Avg Frame rate:	23.976023976023978
tt2437712.mp4 download successful
5. Video ID:	tt2437712	 URL: 	 https://www.youtube.com/watch?v=H3DM-HHJNyc	Codec:	h264	Resolution:	1280 x 720	Avg Frame rate:	23.97602352760927
8. Video ID:	tt2243537	 URL: 	 https://www.imdb.com/videoplayer/vi3723732761	Codec:	h264	Resolution:	854 x 480	Avg Frame rate:	29.97
9. Video ID:	tt1980209	 URL: 	 https://www.imdb.com/videoplayer/vi3975259673	Codec:	h264	Resolution:	852 x 480	Avg Frame rate:	23.976145560761882
13. Video ID:	tt1741243	 URL: 	 https://www.imdb.com/videoplayer/vi2169153049	Codec:	h264	Resolution:	534 x 360	Avg Frame rate:	29.97
tt1723121.mp4 download successful
15. Video ID:	tt1723121	 URL: 	 https://www.youtube.com/watch?v=O7NHfAzg7Yg	Codec:	

In [ ]:
len(missingVideos)

0

### Generate 1min scenes

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import subprocess

SCENES_DIR = os.path.join(TEST_DIR,'scenes')

def split_video(file):
    input_file= os.path.join(DATASET_DIR,file)
    output_dir=os.path.join(SCENES_DIR,file.split('.')[0])
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    result = subprocess.run(["ffmpeg", "-i", input_file,"-map","0", "-c", "copy", "-f", "segment", "-segment_time", "60","-reset_timestamps", "1", output_dir+"/"+file.split(".")[0]+".%02d."+file.split(".")[-1]],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    
for video in os.listdir(DATASET_DIR):
    print(video)
    split_video(video)

tt2437712.mp4
tt2788710.mp4
tt2872718.mp4
tt2243537.mp4
tt1723121.mp4
tt1980209.mp4
tt1741243.mp4
tt1655460.mp4
tt1308728.mp4
tt0465602.mp4
tt1247667.mp4
PGuqnE35cCg.mp4
tt0489019.mp4
PFs7mxKNUO8.mp4
KPMNAjV3UyY.mp4
IvC8zu0ixNo.mp4
i8KnemK3YTM.mp4
FVLQM61SIf0.mp4
F9bx6k7-hk0.mp4
eF2Vms5WLPc.mp4
DODLEX4zzLQ.mp4
dajfpdtPGvs.mp4
bZKpc2WCBvs.mp4
bWxQPzpL2zs.mp4
bcENL_Vdoqw.mp4
Auyj3CNF5xk.mp4
30-LwQM9IlY.mp4
YYlPKnlpJ2U.mp4
xfsGIrvglO8.mp4
X27R97m0Z8c.mp4
vZMd_PSEuRk.mp4
VxlUTNfoBPw.mp4
tt7690016.mp4
tt6521876.mp4
tt6051948.mp4
tt4843640.mp4
tt4651520.mp4
tt4158096.mp4
tt3675748.mp4
tt3960412.mp4
SsIm6bRUg7M.mp4
tt2937696.mp4
tt3491962.mp4
sra_H9o7pjA.mp4
SPmxsRDSmTc.mp4
7XkWU0M0M9s.mp4
4Xp4z5qlyqs.mp4
0g_aiwRKJqM.mp4


### Preparing Test Dataset

In [10]:
import os
import shutil
import pandas as pd

SCENES_DIR = os.path.join(TEST_DIR,'scenes')


scenesDirectory= SCENES_DIR
testDataDirectory= os.path.join(TEST_DIR,'TestDataset')

if not os.path.exists(testDataDirectory):
    os.makedirs(testDataDirectory)

totalSuccessfullAttempts=0
totalFailedAttempts=0
for i,videoID in enumerate(csvFile["Video ID"]):
    try:
        sceneID = str(csvFile["Scene_ID"][i])
        if int(sceneID) < 10:
            shutil.copyfile(os.path.join(scenesDirectory,videoID,f'{videoID}.0{sceneID}.mp4'),os.path.join(testDataDirectory,f'{videoID}.0{sceneID}.mp4'))
        else:
            shutil.copyfile(os.path.join(scenesDirectory,videoID,f'{videoID}.{sceneID}.mp4'),os.path.join(testDataDirectory,f'{videoID}.{sceneID}.mp4'))

        # shutil.copyfile(scenesDirectory+videoID+"/"+videoID+".0"+sceneID+".mp4", trainingDataDirectory+videoID+".0"+sceneID+".mp4")
        print("Video ID:",videoID,"Scene ID:",sceneID, "Successfully extracted")
        totalSuccessfullAttempts+=1
    except Exception as e:
        print("Video ID:",videoID,"Scene ID:",sceneID, "Failed")
        print('Exception : ',e)
        totalFailedAttempts+=1
print("Total Successfull Scene Extraction Attempts = ", totalSuccessfullAttempts)
print("Total Failed Scene Extraction Attempts = ", totalFailedAttempts)

Video ID: tt2872718 Scene ID: 0 Successfully extracted
Video ID: tt2872718 Scene ID: 1 Successfully extracted
Video ID: tt2788710 Scene ID: 0 Successfully extracted
Video ID: tt2788710 Scene ID: 1 Successfully extracted
Video ID: tt2788710 Scene ID: 2 Successfully extracted
Video ID: tt2437712 Scene ID: 0 Successfully extracted
Video ID: tt2437712 Scene ID: 1 Successfully extracted
Video ID: tt2437712 Scene ID: 2 Successfully extracted
Video ID: tt2243537 Scene ID: 0 Successfully extracted
Video ID: tt1980209 Scene ID: 0 Successfully extracted
Video ID: tt1980209 Scene ID: 1 Successfully extracted
Video ID: tt1980209 Scene ID: 2 Successfully extracted
Video ID: tt1980209 Scene ID: 3 Successfully extracted
Video ID: tt1741243 Scene ID: 0 Successfully extracted
Video ID: tt1741243 Scene ID: 1 Successfully extracted
Video ID: tt1723121 Scene ID: 0 Successfully extracted
Video ID: tt1723121 Scene ID: 1 Successfully extracted
Video ID: tt1723121 Scene ID: 2 Successfully extracted
Video ID: 